In [1]:
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print(tf.__version__)

2.2.0


In [70]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        desired_accuracy=0.99
        if(logs.get('val_accuracy')>desired_accuracy):
            print(f"\nReached {desired_accuracy} accuracy so cancelling training!")
            self.model.stop_training = True

In [3]:
labels=[]
images=[]

with open('/tmp/sign/sign_mnist_train.csv') as train_file:
    reader=csv.reader(train_file)
    next(reader)

    for row in reader:
        labels.append(int(row[0]))
        image=[]
        for i in range(1, 784+1):
            image.append(int(row[i]))
        images.append(image)
        
labels = np.array(labels)
images = np.array(images)
images=images.reshape(-1, 28, 28, 1)

In [21]:
images.shape

(27455, 28, 28, 1)

In [4]:
valid_images=[]
valid_labels=[]
with open('/tmp/sign/sign_mnist_test.csv') as  valid_file:
    reader=csv.reader(valid_file)
    next(reader)
    for row in reader:
        valid_labels.append(int(row[0]))
        image=[]
        for i in range(1, 784+1):
            image.append(int(row[i]))
        valid_images.append(image)
        
valid_labels = np.array(valid_labels)        
valid_images = np.array(valid_images)
valid_images=valid_images.reshape(-1, 28, 28, 1)

In [20]:
valid_images.shape

(7172, 28, 28, 1)

In [60]:
train_gen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
)
train_generator = train_gen.flow(images, labels)

In [47]:
valid_gen = ImageDataGenerator(rescale=1/255)
valid_generator = valid_gen.flow(valid_images, valid_labels)

In [71]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.BatchNormalization(),
      
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(25, activation='softmax'),
])

In [72]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 13, 13, 16)        0         
_________________________________________________________________
batch_normalization_27 (Batc (None, 13, 13, 16)        64        
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 5, 5, 32)          0         
_________________________________________________________________
batch_normalization_28 (Batc (None, 5, 5, 32)          128       
_________________________________________________________________
flatten_11 (Flatten)         (None, 800)             

In [73]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [74]:
model.fit(train_generator, epochs=100, validation_data=valid_generator, 
    callbacks=[myCallback(), tf.keras.callbacks.ModelCheckpoint("/tmp/sign/best.h5", save_best_only=True, verbose=2)]
    )

Epoch 1/100
856/858 [============================>.] - ETA: 0s - loss: 0.9276 - accuracy: 0.7244
Epoch 00001: val_loss improved from inf to 0.43674, saving model to /tmp/sign/best.h5
858/858 [==============================] - 7s 9ms/step - loss: 0.9267 - accuracy: 0.7246 - val_loss: 0.4367 - val_accuracy: 0.8454
Epoch 2/100
853/858 [============================>.] - ETA: 0s - loss: 0.2836 - accuracy: 0.9132
Epoch 00002: val_loss improved from 0.43674 to 0.18962, saving model to /tmp/sign/best.h5
858/858 [==============================] - 8s 9ms/step - loss: 0.2832 - accuracy: 0.9131 - val_loss: 0.1896 - val_accuracy: 0.9264
Epoch 3/100
854/858 [============================>.] - ETA: 0s - loss: 0.1745 - accuracy: 0.9472
Epoch 00003: val_loss did not improve from 0.18962
858/858 [==============================] - 7s 9ms/step - loss: 0.1743 - accuracy: 0.9473 - val_loss: 0.5417 - val_accuracy: 0.8410
Epoch 4/100
853/858 [============================>.] - ETA: 0s - loss: 0.1209 - accuracy: